# Data Visualization

## Libraries

In [1]:
from preprocessing import load_and_prepare_model, run_predict
import random
import os

# Taxon Class

In [2]:
class Taxon:
    def __init__(self, name, rank):
        self.name = name
        self.rank = rank
        self.parent = None
        self.children = []

    def set_parent(self, parent):
        """Sets the parent of this taxon and adds this taxon as a child to the parent."""
        self.parent = parent
        parent.add_child(self)

    def add_child(self, child):
        """Adds a child taxon to this taxon."""
        self.children.append(child)

    def __str__(self):
        """String representation of the taxon."""
        return f"{self.rank}: {self.name}"

class TaxonomyManager:
    def __init__(self):
        self.taxa = {}  # Stores all taxa regardless of rank

    def add_taxon(self, name, rank, parent_name=None):
        """Adds a taxon to the taxonomy."""
        taxon = Taxon(name, rank)
        self.taxa[name] = taxon
        if parent_name:
            parent = self.taxa.get(parent_name)
            if parent:
                taxon.set_parent(parent)

    def get_taxon(self, name):
        """Retrieves a taxon by its name."""
        return self.taxa.get(name)

    def display_taxonomy(self):
        """Displays the entire taxonomy in a hierarchical format."""
        # Find roots (taxa without parents)
        roots = [taxon for taxon in self.taxa.values() if taxon.parent is None]
        for root in roots:
            self._display_taxon(root, 0)

    def _display_taxon(self, taxon, indent):
        """Recursive helper function to display a taxon and its children."""
        print(' ' * indent + str(taxon))
        for child in taxon.children:
            self._display_taxon(child, indent + 4)

# Populate Taxonomy

In [3]:
taxonomy = TaxonomyManager()

# Populate Objects
object_names = ["marine life", "inanimate"]
for name in object_names:
    taxonomy.add_taxon(name, "Binary")

# Populate Binaries
binary_mapping = {
    "asteroidea": "marine life",
    "phaeophyceae": "marine life",
    "bivalia": "marine life",
    "myxini": "marine life",
    "artificial": "inanimate",
    "natural": "inanimate",
    "chlorophyta": "marine life",
    "monocots": "marine life"
}
for class_name, binary_name in binary_mapping.items():
    taxonomy.add_taxon(class_name, "Class", binary_name)

# Populate Classes
class_mapping = {
    "asterias": "asteroidea",
    "fucus": "phaeophyceae",
    "henrica": "asteroidea",
    "mya": "bivalia",
    "myxine": "myxini",
    "cylindrical": "artificial",
    "solid": "natural",
    "arboral": "natural",
    "saccharina": "phaeophyceae",
    "ulva": "chlorophyta",
    "urospora": "chlorophyta",
    "zostera": "monocots"
}
for genus_name, class_name in class_mapping.items():
    taxonomy.add_taxon(genus_name, "Genus", class_name)

# Populate Genera
genus_mapping = {
    "asterias rubens": "asterias",
    "asteroidea": "asterias",
    "fucus vesiculosus": "fucus",
    "henrica": "henrica",
    "mytilus edulis": "mya",
    "myxine glurinosa": "myxine",
    "pipe": "cylindrical",
    "rock": "solid",
    "saccharina latissima": "saccharina",
    "tree": "arboral",
    "ulva intestinalis": "ulva",
    "urospora": "urospora",
    "zostera marina": "zostera"
}
for species_name, genus_name in genus_mapping.items():
    taxonomy.add_taxon(species_name, "Species", genus_name)

# Model Hooks

# Extract Prediction from Yolo v8

In [4]:
# Load the model with random weights
random.seed(133745)
IMGDIR_PATH = "/mnt/RAID/datasets/label-studio/fjord/images/"
imgpath = IMGDIR_PATH + random.choice(os.listdir(IMGDIR_PATH))
model, hooks = load_and_prepare_model("runs/segment/Yolov8n-seg-train/weights/best.pt")
results = run_predict(imgpath, model, hooks, score_threshold=0.5, iou_threshold=0.7)


WARNING ⚠️ NMS time limit 0.550s exceeded
image 1/1 /mnt/RAID/datasets/label-studio/fjord/images/frame013780.jpg: 640x576 1 ulva intestinalis, 1 urospora, 1 zostera marina, 331.0ms
Speed: 12.0ms preprocess, 331.0ms inference, 1758.8ms postprocess per image at shape (1, 3, 640, 576)


In [5]:
len(results)

3

In [6]:
len(results[0]['activations'])

13